## Wordnet
#### Sources
* [Blog](https://blog.xrds.acm.org/2017/07/power-wordnet-use-python/#:~:text=WordNet%20is%20a%20database%20of,and%20sat%20on%20the%20sofa.%E2%80%9D)
* [Theory](https://wordnet.princeton.edu/)
* [Medium article on query relaxation](https://queryunderstanding.com/query-relaxation-342bc37ad425)
* [ Paper: Similarity Search Combining Query Relaxation and Diversification](https://arxiv.org/ftp/arxiv/papers/1611/1611.04689.pdf)
* [Similarity application example](https://stackoverflow.com/questions/15730473/wordnet-find-synonyms)

In [115]:
from nltk.corpus import wordnet as wn

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [116]:
A = wn.synsets('capture')

In [85]:
A[0].hypernyms()[0].lemma_names()

['acquiring', 'getting']

In [106]:
wn.synsets('fender')[1].lemma_names()

['fender', 'buffer', 'cowcatcher', 'pilot']

In [98]:
def extract_hypernym(query_term):
    hypernym_list = {}
    contexts = {}
    for i, element in enumerate(wn.synsets(query_term)):
        temp = element.hypernyms()[0].lemma_names()
        contexts[i] = temp 
    return contexts


In [99]:
extract_hypernym('pacification')

{0: ['appeasement', 'calming'],
 1: ['treaty', 'pact', 'accord'],
 2: ['conflict', 'struggle', 'battle']}

In [107]:
def extract_synonyms(query_term):
    temp = {}
    contexts = {}
    for i, element in enumerate(wn.synsets(query_term)):
        temp = element.lemma_names()
        contexts[i] = temp
    return contexts

In [114]:
extract_synonyms("cacophony")

{0: ['blare', 'blaring', 'cacophony', 'clamor', 'din'], 1: ['cacophony']}